In [5]:
import numpy as np
import cv2

In [30]:
# read video file
cap = cv2.VideoCapture("test_videos\\solidYellowLeft.mp4")
# generate video file for output video
frame_width = 960    #frame width of your cam
frame_height = 540   #frame height of your cam 
out = cv2.VideoWriter('out_solidYellowLeft.mp4',cv2.VideoWriter_fourcc('M','J','P','G'), 30, (frame_width,frame_height))

while True:
    ret, frame = cap.read()
    if ret == True:
        # Convert frame to gray scale
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        # Apply Gaussian Blur
        kernel_size = 5  #kernal size of Gaussian Blur
        blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0) 
        # Apply Canny Edges
        low_threshold = 50   #low threshold
        high_threshold = 150  #high threshold
        edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
        # Apply mask and Region of Interest
        mask = np.zeros_like(edges)   
        ignore_mask_color = 255  
        imshape = frame.shape
        vertices = np.array([[(50,imshape[0]),(450, 320), (500, 320), (900,imshape[0])]], dtype=np.int32)
        cv2.fillPoly(mask, vertices, ignore_mask_color)
        masked_edges = cv2.bitwise_and(edges, mask)
        
        # Apply Hough 
        rho = 2  # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 15     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = 40 #minimum number of pixels making up a line
        max_line_gap = 20    # maximum gap in pixels between connectable line segments
        line_image = np.copy(frame)*0 # creating a blank to draw lines on

        # Run Hough on edge detected image
        # Output "lines" is an array containing endpoints of detected line segments
        lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
        
        # Iterate over the output "lines" and draw lines on a blank image
        x_left = []
        x_right = []
        
        y_left = []
        y_right = []
        
        for line in lines:
            for x1,y1,x2,y2 in line:
                if x1 < imshape[1]/2 and x2 < imshape[1]/2:
                    # Collecting all x and y values from left part of image 
                    x_left += [x1, x2]
                    y_left += [y1, y2]
                else:
                    # Collecting all x and y values from right part of image
                    x_right += [x1, x2]
                    y_right += [y1, y2]
                    
        if len(x_left) != 0 and len(y_left) != 0:
            # Calculating average slope and b from all the points on left - here slope will be negative
            m,b = np.polyfit(x_left, y_left, 1)
            min_x = min(x_left)
            max_x = max(x_left)
            # Plotting line from x1=(y_max-b)/m, y1=y_max to x2=x_max, y2=m*x_max + b
            cv2.line(line_image, (int((imshape[0]-b)/m), imshape[0]), (max_x, int(max_x * m + b)), (0,0,255), 10)
        if len(x_right) != 0 and len(y_right) != 0:
            # Calculating average slope and b from all the points on right - here slope will be positive
            m,b = np.polyfit(x_right, y_right, 1)
            min_x = min(x_right)
            max_x = max(x_right)
            # Plotting line from x1=x_min, y1=(x_min*m)+b to x2=(y_max-b)/m, y2=y_max
            cv2.line(line_image, (min_x, int(min_x * m + b)), (int((imshape[0]-b)/m), imshape[0]), (0,0,255), 10)
        # Create a "color" binary image to combine with line image
        color_edges = np.dstack((edges, edges, edges)) 
        # Draw the lines on the edge image
        lines_edges = cv2.addWeighted(frame, 0.8, line_image, 1, 0)
        # Show image after detecting lane lines
        cv2.imshow("frame",lines_edges)
        # Save output frame to output video
        out.write(lines_edges)
        #click on q button to close windows
        if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == ord('e'):
            break
    else:
        break

# Release cap and out        
cap.release()
out.release()
cv2.destroyAllWindows()  # Close all windows